In [24]:
import torch
import collections
import matplotlib.pyplot as plt
import numpy as np
import random
import torch.nn.functional as F
import pandas as pd

In [3]:
import sys
sys.path.insert(1, '/home/buehlern/Documents/Masterarbeit/models')
from src.data.mri_datamodule import MRIDataModule
from src.models.vit_mae_module import VisionTransformerMAE

/home/buehlern/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load model checkpoint
mae_checkpoint = "/home/buehlern/Documents/Masterarbeit/models/checkpoints/ViT-L MAE.ckpt"
checkpoint = torch.load(mae_checkpoint)
state_dict = checkpoint['state_dict']

/tmp/ipykernel_638206/1545113530.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(mae_checkpoint)
/home/buehlern/.local/lib/python3.10/site-packag

In [5]:
state_dict.keys()

odict_keys(['net._orig_mod.vit.embeddings.cls_token', 'net._orig_mod.vit.embeddings.position_embeddings', 'net._orig_mod.vit.embeddings.patch_embeddings.projection.weight', 'net._orig_mod.vit.embeddings.patch_embeddings.projection.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.query.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.query.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.key.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.key.bias', 'net._orig_mod.vit.encoder.layer.0.attention.attention.value.weight', 'net._orig_mod.vit.encoder.layer.0.attention.attention.value.bias', 'net._orig_mod.vit.encoder.layer.0.attention.output.dense.weight', 'net._orig_mod.vit.encoder.layer.0.attention.output.dense.bias', 'net._orig_mod.vit.encoder.layer.0.intermediate.dense.weight', 'net._orig_mod.vit.encoder.layer.0.intermediate.dense.bias', 'net._orig_mod.vit.encoder.layer.0.output.dense.weight', 'net._orig_mod.vit.encoder.layer.0.outp

In [6]:
def remap_keys(state_dict, unwanted_prefix, new_prefix):
    new_state_dict = {}
    for key, value in state_dict.items():
        if key.startswith(unwanted_prefix):
            new_key = key.replace(unwanted_prefix, new_prefix)
        else:
            new_key = key
        new_state_dict[new_key] = value
    return collections.OrderedDict(new_state_dict)

In [7]:
new_state_dict = remap_keys(state_dict, 'net._orig_mod.', 'net.')

In [8]:
new_state_dict.keys()

odict_keys(['net.vit.embeddings.cls_token', 'net.vit.embeddings.position_embeddings', 'net.vit.embeddings.patch_embeddings.projection.weight', 'net.vit.embeddings.patch_embeddings.projection.bias', 'net.vit.encoder.layer.0.attention.attention.query.weight', 'net.vit.encoder.layer.0.attention.attention.query.bias', 'net.vit.encoder.layer.0.attention.attention.key.weight', 'net.vit.encoder.layer.0.attention.attention.key.bias', 'net.vit.encoder.layer.0.attention.attention.value.weight', 'net.vit.encoder.layer.0.attention.attention.value.bias', 'net.vit.encoder.layer.0.attention.output.dense.weight', 'net.vit.encoder.layer.0.attention.output.dense.bias', 'net.vit.encoder.layer.0.intermediate.dense.weight', 'net.vit.encoder.layer.0.intermediate.dense.bias', 'net.vit.encoder.layer.0.output.dense.weight', 'net.vit.encoder.layer.0.output.dense.bias', 'net.vit.encoder.layer.0.layernorm_before.weight', 'net.vit.encoder.layer.0.layernorm_before.bias', 'net.vit.encoder.layer.0.layernorm_after.wei

In [9]:
mae = VisionTransformerMAE(image_size = 3072, patch_size = 48)
mae.load_state_dict(new_state_dict)

<All keys matched successfully>

In [10]:
mae.eval()

VisionTransformerMAE(
  (net): ViTMAEForPreTraining(
    (vit): ViTMAEModel(
      (embeddings): ViTMAEEmbeddings(
        (patch_embeddings): ViTMAEPatchEmbeddings(
          (projection): Conv2d(1, 768, kernel_size=(48, 48), stride=(48, 48))
        )
      )
      (encoder): ViTMAEEncoder(
        (layer): ModuleList(
          (0-23): 24 x ViTMAELayer(
            (attention): ViTMAEAttention(
              (attention): ViTMAESelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): ViTMAESelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (intermediate): ViTMAEInter

In [43]:
# Disable masking
#mae.net.config.mask_ratio = 0

In [12]:
# For outputting attentions
print('output_attentions:', mae.net.config.output_attentions)
print('_attn_implementation:', mae.net.config._attn_implementation)

output_attentions: True
_attn_implementation: eager


In [13]:
# Load the DataModule
mri_datamodule = MRIDataModule(image_size = 3072, square = True, output_channels = 1, cache = False, fix_inverted = True)

initializing MRIDatasetBase ...
reading /home/buehlern/Documents/Masterarbeit/data/clean_df_slim_frac.pkl file ...
PATH /home/buehlern/Documents/Masterarbeit/data/BodyPartExamined_mappings_mergemore.json
/home/buehlern/Documents/Masterarbeit/data/cache-full/df_labelcomparison.pkl does not exit --> no items excluded by it
MRIDatasetBase(len=639877) initialized

initializing MRIDataset(mode=train) ...
MRIDataset(mode=train, len=516402) initialized

initializing MRIDataset(mode=val) ...
MRIDataset(mode=val, len=27518) initialized

initializing MRIDataset(mode=test) ...
WARN: including test data
MRIDataset(mode=test, len=95957) initialized


In [14]:
def show_image(image, title=''):
    # image is [H, W, 1]
    assert image.shape[2] == 1
    plt.imshow(image, cmap=plt.cm.bone)
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

In [15]:
dl_iter = iter(mri_datamodule.data_val)

In [58]:
def visualize(pixel_values, model, imgname=None):
    image_size = model.config.image_size
    patch_size = model.config.patch_size
    num_patches = image_size // patch_size
    
    # forward pass
    outputs = model(pixel_values, output_attentions=True)
    y = model.unpatchify(outputs.logits)
    y = torch.einsum('nchw->nhwc', y).detach().cpu()
    
    # visualize the mask
    mask = outputs.mask.detach()
    mask = mask.unsqueeze(-1).repeat(1, 1, model.config.patch_size**2 *1)  # (N, H*W, p*p*1)
    mask = model.unpatchify(mask)  # 1 is removing, 0 is keeping
    mask = torch.einsum('nchw->nhwc', mask).detach().cpu()
    
    x = torch.einsum('nchw->nhwc', pixel_values)

    # masked image
    im_masked = x * (1 - mask)

    # MAE reconstruction pasted with visible patches
    im_paste = x * (1 - mask) + y * mask

    # Attention map calculations
    # seq_len = 1025
    # Without [CLS] token: seq_len = 1024
    # num_patches_x_out = num_patches_y = sqrt(seq_len) = sqrt(1024) = 32
    # num_patches_x_in = image_size // patch_size = 3072 // 48 = 64
    # Reason for difference is masking! 4096 * (1-0.75) = 1024

    #image_size = model.config.image_size
    #patch_size = model.config.patch_size
    #num_patches = image_size // patch_size
    #attn_map = attn.squeeze().numpy()
    ## Scale attention map to original image size
    #attn_map_scaled = np.zeros((image_size, image_size))
    #for i in range(num_patches):
    #    for j in range(num_patches):
    #        x_start = i * patch_size
    #        x_end = x_start + patch_size
    #        y_start = j * patch_size
    #        y_end = y_start + patch_size
    #        
    #        attn_map_scaled[x_start:x_end, y_start:y_end] = attn_map[i * num_patches + j]
    
    #num_patches = attn.size(-1)
    #print(f"num_patches (after masking {model.config.mask_ratio}):", num_patches) # 1024
    #attn = attn[0].reshape(model.config.patch_size, model.config.patch_size, -1)
    # Rescale attention to image size
    #attn = F.interpolate(attn.unsqueeze(0).unsqueeze(0), scale_factor=model.config.patch_size, mode="nearest")[0][0]

    attentions = outputs.attentions
    print("len(attentions):", len(attentions)) # 24 layers
    print("attentions[0].shape:", attentions[0].shape) # [1, 16, 1025, 1025]
    # Initialize full attention map
    full_attn_map = torch.zeros((1, image_size, image_size)) # (1, 3072, 3072)
    masklist = outputs.mask.detach().type(torch.int64) > 0
    print("masklist.shape:", masklist.shape) # (1, 4096)
    # Determine patch contributions
    #patch_contrib = attentions[-1][:, :, 1:, 1:].mean(dim=1) # Take mean attention of last layer (without [CLS] token)
    patch_contrib = torch.zeros(attentions[0].shape[-1]-1) # -1 for [CLS] token
    check_cls_token = False
    if check_cls_token:
        for layer_attn in attentions:
            # Only check [CLS] token
            attn = layer_attn[:, :, 1, 1:].detach().cpu() # [1, 16, 1024]
            # Average over heads
            attn = attn.mean(dim=1) # [1, 1024]
            # Average over batch (if batch_size > 1)
            attn = attn.mean(dim=0) # [1, 1024]
            patch_contrib += attn
    else:
        for layer_attn in attentions:
            # Remove [CLS] token
            attn = layer_attn[:, :, 1:, 1:].detach().cpu() # [1, 16, 1024, 1024]
            # Average over heads
            attn = attn.mean(dim=1) # [1, 1024, 1024]
            # Average over batch (if batch_size > 1)
            attn = attn.mean(dim=0) # [1024, 1024]
            # Average contribution across all other tokens
            attn = attn.mean(dim=1) # [1024,]
            patch_contrib += attn
    # Normalize
    patch_contrib -= patch_contrib.min()
    patch_contrib /= patch_contrib.max()
    print("patch_contrib.shape:", patch_contrib.shape)
    print(pd.DataFrame(patch_contrib).describe())
    # Map attention scores onto full attention map
    attn_iter = iter(patch_contrib)
    print("len(masklist[0]):", len(masklist[0]))
    for i, masked in enumerate(masklist[0]):
        row = i // num_patches
        col = i % num_patches
        #print("i, row, col, masked:", i, row, col, masked.item())
        attn_val = next(attn_iter) if ~masked else torch.tensor([0])
        full_attn_map[0, row*patch_size:(row+1)*patch_size, col*patch_size:(col+1)*patch_size] = attn_val.expand((patch_size, patch_size))
    
    # make the plt figure larger
    plt.rcParams['figure.figsize'] = [24, 10]

    plt.subplot(1, 5, 1)
    show_image(x[0], "original")

    plt.subplot(1, 5, 2)
    show_image(im_masked[0], "masked")

    plt.subplot(1, 5, 3)
    show_image(y[0], f"reconstruction (loss: {outputs.loss.item():.4f})")

    plt.subplot(1, 5, 4)
    show_image(im_paste[0], "reconstruction + visible")
    
    plt.subplot(1, 5, 5)
    plt.imshow(full_attn_map[0], cmap='gray', interpolation='nearest')
    plt.title("Attention Map")
    plt.axis('off')

    if imgname is not None:
        plt.savefig('/home/buehlern/Documents/Masterarbeit/notebooks/Data Exploration Graphics/Model Eval/ViT MAE/' + str(imgname) + '.png')
    plt.show()

In [ ]:
item = next(dl_iter)
image = item[0]
batch = image.unsqueeze(0)
visualize(batch, mae.net, imgname="example")

In [ ]:
#shuf_dl_iter = iter(random.sample(list(mri_datamodule.data_val), len(mri_datamodule.data_val)))
#for i in range(50):
#    item = next(shuf_dl_iter)
#    image = item[0]
#    batch = image.unsqueeze(0)
#    visualize(batch, mae.net, imgname=i)